# SQL Server Agent - Detailed job history

This notebook blends sql and python to accomplish transformations with pandas and pulling in file metadata with python.  All of this creates an ending dataframe with comprehensive information in one place about each stored procedure run by each job and what target is being updated.

In [3]:
import pyodbc
import pandas as pd
import os 
from sqlalchemy import create_engine
import urllib
import re

# Setting this option to evaluate the code stored in a field
pd.set_option('display.max_colwidth', None)
# Database connection details
server = '#########'  
database = '#########'  
username = '#########'  
password = '#########'


# Construct the connection string
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
encoded_conn_str = urllib.parse.quote_plus(connection_string)

# Create the SQLAlchemy engine 
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={encoded_conn_str}')

#
query = '''
SELECT 
    SCHEMA_NAME(p.schema_id) as SchemaName
    , p.name as ProcedureName
    , m.definition as ProcedureDefinition
FROM sys.procedures as p
inner join sys.sql_modules as m
on p.object_id = m.object_id
'''
df = pd.read_sql(query, engine)




In [4]:
# need regex to process raw text for stored procedure stored in df.ProcedureDefinition
# df.ProcedureDefinition.head()

In [5]:
# Define a helper to extract ETL info from stored procedure SQL
def parse_etl_info_from_sql(proc_name, sql_text):
    etl_info_list = []

    # Detect all BULK INSERT operations
    bulk_inserts = re.findall(r"BULK\s+INSERT\s+(#\w+)\s+FROM\s+'([^']+)'", sql_text, re.IGNORECASE)

    # Detect all MERGE operations
    merges = re.findall(r"MERGE\s+(\[.*?\])\s+as\s+tgt\s+USING\s+(#\w+)", sql_text, re.IGNORECASE)

    # Detect all TRUNCATE TABLE operations
    truncates = re.findall(r"TRUNCATE\s+TABLE\s+(\[.*?\])", sql_text, re.IGNORECASE)

    # Detect all INSERT INTO ... SELECT * FROM #temp_table
    inserts = re.findall(r"INSERT\s+INTO\s+(\[.*?\])\s+SELECT\s+\*\s+FROM\s+(#\w+)", sql_text, re.IGNORECASE)

    # Process bulk -> insert flow
    for temp_table, file_path in bulk_inserts:
        # See if that temp table is later used in a MERGE or INSERT
        targets = [target for target, source in merges + inserts if source.lower() == temp_table.lower()]
        for target in targets:
            etl_info_list.append({
                'stored_procedure': proc_name,
                'source': file_path,
                'source_type': 'file',
                'target': target.replace('[', '').replace(']', ''),
                'process_type': 'merge-join' if (target, temp_table) in merges else 'truncate and load'
            })

    return etl_info_list



# Extract ETL info
etl_info_records = []
for idx, row in df.iterrows():
    records = parse_etl_info_from_sql(row['ProcedureName'], row['ProcedureDefinition'])
    etl_info_records.extend(records)

# Convert to DataFrame
df_etl_summary = pd.DataFrame(etl_info_records)


In [6]:
# Checking that we got clean information from the array of stored procedures
# df_etl_summary.head()

In [7]:
# Adding source file metadata to the dataframe

from datetime import datetime

def get_file_info(file_path):
    try:
        stats = os.stat(file_path)
        return {
            'last_modified': datetime.fromtimestamp(stats.st_mtime),
            'last_accessed': datetime.fromtimestamp(stats.st_atime),
            'file_size_mb': round(stats.st_size / (1024 * 1024), 2)
        }
    except FileNotFoundError:
        return {
            'last_modified': None,
            'last_accessed': None,
            'file_size_mb': None
        }

# Step 1: Replace vault path with mapped drive
df_etl_summary['server_path'] = df_etl_summary['source'].str.replace(
    '\\\\vault\\powerbiflatfiles$\\production\\', 'V:\\Production\\', regex=False
)

# Step 2: Normalize file paths
df_etl_summary['server_path'] = df_etl_summary['server_path'].apply(os.path.normpath)

# Step 3: Apply the file info function
file_info_list = df_etl_summary['server_path'].apply(get_file_info)

# Step 4: Convert to DataFrame
df_file_info = pd.DataFrame(file_info_list.tolist())

# Step 5: Merge with original summary
df_full = pd.concat([df_etl_summary, df_file_info], axis=1)


# df_full.head()

In [8]:
# Memory efficient approach to get row and column count from files
def get_delimited_file_shape(file_path):
    try:
        # Detect delimiter based on extension
        delimiter = ',' if file_path.lower().endswith('.csv') else '|'
        
        # Open file in text mode
        with open(file_path, 'r', encoding='utf-8') as f:
            header = f.readline()
            col_count = len(header.strip().split(delimiter))
            row_count = sum(1 for _ in f)  # count remaining lines (i.e., rows)
        
        return (row_count, col_count)
    except Exception:
        return (None, None)

In [9]:

df_full[['n_rows', 'n_cols']] = df_full['server_path'].apply(
    lambda x: pd.Series(get_delimited_file_shape(x))
)

In [10]:
# Results look good and the runtime was also quick
# df_full.head()

In [11]:
# df_full.to_sql('stored_procedures', con=engine, schema='src', if_exists='replace', index=False)

In [12]:
# Close the connection
engine.dispose() 

In [13]:
# Get the stored procedures run details from msdb
# Database connection details 
server = '#########'  
database = '#########'  
username = '#########'  
password = '#########'


# Construct the connection string
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
encoded_conn_str = urllib.parse.quote_plus(connection_string)

# Create the SQLAlchemy engine 
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={encoded_conn_str}')

query = '''
SELECT  
    [job],
    [step],
    [step_order],
    [start_time],
    [job_outcome],
    [duration_in_seconds],
    [error_message],
    [command]
FROM [msdb].[dbo].[job_step_history_analysis] AS a1
WHERE 
    [job] NOT IN ('syspolicy_purge_history', 'backup db - ucr_health')
    AND [start_time] = (
        SELECT MAX(a2.[start_time])
        FROM [msdb].[dbo].[job_step_history_analysis] AS a2
        WHERE a2.[step] = a1.[step]
          AND a2.[job] = a1.[job]
    );
'''
df = pd.read_sql(query, engine)

# Close the connection
engine.dispose() 

In [14]:
# results came out as expected
# df.head()

In [15]:
# Using regex to process the command to get the stored procedure to join to the other dataframe

df['stored_procedure'] = (
    df['command']
    .str.replace(r'[\[\];]', '', regex=True)              # remove brackets and semicolon
    .str.extract(r'\.([^.]+)\s*$', expand=False)          # get string after last dot
)
df.stored_procedure.head()

0    update_fact_smart_tools_log
1           update_fact_smartset
2                 shrink_temp_db
3                 shrink_temp_db
4        update_src_availability
Name: stored_procedure, dtype: object

In [16]:
# Merging df_full and df on stored_procedure

df_joined = pd.merge(
    df_full,
    df,
    on='stored_procedure',
    how='inner'
)

In [17]:
# Results are as expected
# df_joined.head()

In [18]:
# Connect back to main database to write the table to the server

# Database connection details
server = '#########'  
database = '#########'  
username = '#########'  
password = '#########'


# Construct the connection string
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
encoded_conn_str = urllib.parse.quote_plus(connection_string)

# Create the SQLAlchemy engine 
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={encoded_conn_str}')

In [19]:
# Write the table to the server

df_joined.to_sql('stored_procedures', con=engine, schema='src', if_exists='replace', index=False)

43

In [20]:
# Close the connection
engine.dispose() 